### OPEN API 를 이용한 챗봇 시스템 

In [1]:
!pip show openapi
# !pip install openai

Name: openapi
Version: 2.0.0
Summary: Python OpenAPI 2.0 (Swagger) object model
Home-page: https://github.com/globality-corp/openapi
Author: Globality Engineering
Author-email: engineering@globality.com
License: 
Location: /opt/anaconda3/lib/python3.11/site-packages
Requires: inflection, jsonschema
Required-by: 


---
### OPENAI API 호출 예제 

In [2]:
from openai import OpenAI
from pprint import pprint
import os
api_key=os.environ.get("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

client = OpenAI(api_key=api_key)
model = 'gpt-3.5-turbo-1106'

messages = [
    {'role':'system','content': 'You are a helphful assistant.'},
    {'role':'user','content': '군자역 혼잡도 알려줘?'}
]
response = client.chat.completions.create(model=model, messages=messages).model_dump()
pprint(response)

{'choices': [{'finish_reason': 'stop',
              'index': 0,
              'logprobs': None,
              'message': {'content': '죄송합니다, 현재 군자역의 혼잡도 정보는 알려드릴 수 없습니다. 군자역 '
                                     '근처의 역 관리소나 공식 웹사이트에서 실시간 정보를 확인하시는 것을 '
                                     '권장드립니다.',
                          'function_call': None,
                          'role': 'assistant',
                          'tool_calls': None}}],
 'created': 1720085417,
 'id': 'chatcmpl-9hD575rBfhSnyAiaTc7r7jAmM2lbq',
 'model': 'gpt-3.5-turbo-1106',
 'object': 'chat.completion',
 'service_tier': None,
 'system_fingerprint': 'fp_44132a4de3',
 'usage': {'completion_tokens': 82, 'prompt_tokens': 36, 'total_tokens': 118}}


> gpt 는 json type 으로 답을 넘겨준다. 

---
### 

 * [model dump ]
    : 메서드 체이닝 -> 메서드를 연쇄적으로 호출하는 방식 

---
### ai chatbot 을 만들기 위해서는 프롬프트 엔지니어링이 필요하다.

In [3]:
prompt = """
내일 다음 주제로 미래 교육 포럼에서 발표할 발제문을 작성해야합니다.
발제문의 주요 내용을 불릿 기호로 간략히 정리해 주세요.
주제:{agenda}


""".format(agenda="인공지능 시대의 교육의 역활과 의미")
model = 'gpt-3.5-turbo-1106'
response = client.chat.completions.create(
    model= model,
    messages=[{'role':'user','content':prompt}]
).to_dict
pprint(response)



<bound method BaseModel.to_dict of ChatCompletion(id='chatcmpl-9hD58Rx4Q2trfXlep9qvKGWw8iftV', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='- 인공지능 기술의 발전과 교육의 관련성\n- 인공지능이 교육에 미치는 영향과 변화\n- 학교 교육과 인공지능의 융합 가능성\n- 인공지능을 효과적으로 활용한 교육 방법과 도구\n- 교사와 학생의 역할 변화와 대비책\n- 인공지능과 교육의 윤리적 문제들에 대한 대처 전략', role='assistant', function_call=None, tool_calls=None))], created=1720085418, model='gpt-3.5-turbo-1106', object='chat.completion', service_tier=None, system_fingerprint='fp_44132a4de3', usage=CompletionUsage(completion_tokens=152, prompt_tokens=97, total_tokens=249))>


In [4]:
from pprint import pprint

prompt = """
내일 다음 주제로 미래 교육 포럼에서 발표할 발제문을 작성해야합니다.
발제문의 주요 내용을 불릿 기호로 간략히 정리해 주세요.
주제:{agenda}
""".format(agenda="인공지능 시대의 교육의 역할과 의미")

model = 'gpt-3.5-turbo-1106'
response = client.chat.completions.create(
    model=model,
    messages=[{'role': 'user', 'content': prompt}]
)

pprint(response.choices[0].message.content)


('- 인공지능이 발전함에 따라 교육의 역할이 어떻게 변화하는가?\n'
 '- 인공지능이 교육에 미치는 영향은 무엇인가?\n'
 '- 학생들을 준비하고 미래를 대비하는 데 교육이 어떤 역할을 해야 하는가?\n'
 '- 인공지능과 교육의 상호작용이 미래 사회에 어떠한 의미를 갖는가?')


---
### Chater 5 프롬프트 엔지니어링의 핵심 기법 
- few shot
- Cot
-SC
- ToT
- ReAct
-RAG


In [5]:
from pprint import pprint
import json
template = """
당신은 번역함수이며, 반환값은 반드시 JSON 데이터 여야 합니다. 
STEP 별로 작업을 수행하면서 그 결과를 아래의 출력 결과 JSON 포맷에 작성하세요.
STEP-1. 아래 세 개의 백틱으로 구분된 텍스트를 원문 그대로 읽어 올것
STEP-2. 입력받은 텍스트가 긍정적이라면 True를 표기할것
STEP-3. 다음의 말투로 번역할 것: ["지구의 나이는 45억살이야.","세종 대왕은 조선의 위대한 국왕이야."]
```{text}```
---
출력결과 : {{"STEP-1":<입력텍스트>, "STEP-2": <true/false>, "STEP-3":<번역결과>}}
"""
text = "William Shakespeare was an English playwrit, poet and actor.  He is widely regarded as the greatest writer in the English language and the world's pre-eminent dramatist."
template = template.format(text=text)
context = [{"role": "user","content": template}]

model_1 = 'gpt-3.5-turbo-1106'
model_2 = 'gpt-4-0613'
response = client.chat.completions.create(
    model=model_2,
    messages=context,
    temperature=0,
    top_p=0,
    seed=1234
).model_dump()
pprint(json.loads(response['choices'][0]['message']['content']))
# pprint(response.choices[0].message.content)


{'STEP-1': 'William Shakespeare was an English playwrit, poet and actor.  He '
           'is widely regarded as the greatest writer in the English language '
           "and the world's pre-eminent dramatist.",
 'STEP-2': True,
 'STEP-3': ['윌리엄 셰익스피어는 영국의 희곡 작가, 시인, 배우였어.',
            '그는 영어로 가장 위대한 작가로 널리 인정받고 있어, 세계에서 가장 뛰어난 극작가야.']}
